## libraries

In [40]:
import pandas as pd
import numpy as np
import re
import pickle

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

## data

In [41]:
df = pd.read_csv('/content/drive/MyDrive/2022 ADV/data/tokenize_df.csv')
df.head(3)

,txt,label,tokenize_txt,tokenize_txt_bigram
0,입니다 무엇을 도와드릴까요 쇼핑에서 문자가 와서 확인차 연락드렸어요 아 그러십니까 ...,0,"['입니다', '무엇', '을', '도와', '드릴까요', '쇼핑', '에서', '...","['입니다 무엇', '무엇 을', '을 도와', '도와 드릴까요', '드릴까요 쇼핑..."
1,쇼핑 입니다 네 뭐 좀 알아 보려고 전화드렸어요 네 고객님 안녕하세요 무엇을 도와드...,0,"['쇼핑', '입니다', '네', '뭐', '좀', '알', '아', '보', '려...","['쇼핑 입니다', '입니다 네', '네 뭐', '뭐 좀', '좀 알', '알 아'..."
2,행복을 전하는 쇼핑 입니다 방송에서 목걸이 세트를 샀는데요 주문하셨다는 말씀이십니까...,0,"['행복', '을', '전하', '는', '쇼핑', '입니다', '방송', '에서'...","['행복 을', '을 전하', '전하 는', '는 쇼핑', '쇼핑 입니다', '입니..."


In [42]:
df.shape

(55833, 4)

In [43]:
df['label'].value_counts()

0    55310
1      523
Name: label, dtype: int64

In [44]:
tokenized_df = df[['tokenize_txt', 'label']]
tokenized_df.head(3)

,tokenize_txt,label
0,"['입니다', '무엇', '을', '도와', '드릴까요', '쇼핑', '에서', '...",0
1,"['쇼핑', '입니다', '네', '뭐', '좀', '알', '아', '보', '려...",0
2,"['행복', '을', '전하', '는', '쇼핑', '입니다', '방송', '에서'...",0


In [45]:
tokenized_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55833 entries, 0 to 55832
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tokenize_txt  55833 non-null  object
 1   label         55833 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 872.5+ KB


## stopwords

In [46]:
def text_clearing(text):

    hangul = re.compile('[^ ㄱ-ㅣㅏ-ㅣ가-힣]+') # 한글이 아닌 텍스트를 찾음
    
    return hangul.sub('', text).split() # 치환할 문자열, target text

In [47]:
tokenized_df['tokenize_txt'] = tokenized_df['tokenize_txt'].apply(lambda x : text_clearing(x))

In [48]:
tokenized_df.head(3)

,tokenize_txt,label
0,"[입니다, 무엇, 을, 도와, 드릴까요, 쇼핑, 에서, 문자, 가, 와서, 확인, ...",0
1,"[쇼핑, 입니다, 네, 뭐, 좀, 알, 아, 보, 려고, 전화, 드렸, 어요, 네,...",0
2,"[행복, 을, 전하, 는, 쇼핑, 입니다, 방송, 에서, 목걸이, 세트, 를, 샀,...",0


In [49]:
# stopwords
stopwords = ["도", "는", "다", "의", "가", "이", "은", "한", "에", "하", "고", "을", "를", "인", "듯", "과", "와", "네", "들", "듯", "지", "임", "게"]

In [50]:
def del_stop_words(text):

    # 불용어
    stopwords = ["도", "는", "다", "의", "가", "이", "은", "한", "에", "하", "고", "을", "를", "인", "듯", "과", "와", "네", "들", "듯", "지", "임", "게"]
    # 불용어 제거
    results = [text[i] for i in range(len(text)) if text[i] not in stopwords]

    return results

In [51]:
# sample
del_stop_words(['집', '에', '갈래'])

['집', '갈래']

In [52]:
# 불용어 제거
tokenized_df['tokenized_del_stopwords'] = tokenized_df['tokenize_txt'].apply(lambda x : del_stop_words(x))

In [53]:
tokenized_df.head(3)

,tokenize_txt,label,tokenized_del_stopwords
0,"[입니다, 무엇, 을, 도와, 드릴까요, 쇼핑, 에서, 문자, 가, 와서, 확인, ...",0,"[입니다, 무엇, 도와, 드릴까요, 쇼핑, 에서, 문자, 와서, 확인, 차, 연락,..."
1,"[쇼핑, 입니다, 네, 뭐, 좀, 알, 아, 보, 려고, 전화, 드렸, 어요, 네,...",0,"[쇼핑, 입니다, 뭐, 좀, 알, 아, 보, 려고, 전화, 드렸, 어요, 고객, 님..."
2,"[행복, 을, 전하, 는, 쇼핑, 입니다, 방송, 에서, 목걸이, 세트, 를, 샀,...",0,"[행복, 전하, 쇼핑, 입니다, 방송, 에서, 목걸이, 세트, 샀, 는데요, 주문,..."


## encoding

- train : test = 8 : 2 로 분리

In [54]:
tr, te = train_test_split(tokenized_df, test_size=0.2, random_state=0)

In [55]:
tr.head(3)

,tokenize_txt,label,tokenized_del_stopwords
4841,"[네, 쇼핑, 입니다, 무엇, 을, 도와, 드릴까요, 지금, 하, 고, 있, 는, ...",0,"[쇼핑, 입니다, 무엇, 도와, 드릴까요, 지금, 있, 거, 주문, 할, 수, 있,..."
35591,"[롯데, 월드, 를, 가려, 하, 는데요, 방법, 을, 알려, 주, 세요, 출발지,...",0,"[롯데, 월드, 가려, 는데요, 방법, 알려, 주, 세요, 출발지, 어디, 십니까,..."
13194,"[입니다, 제, 가, 얼마, 전, 에, 에서, 셔츠, 를, 주문, 했, 거든요, 네...",0,"[입니다, 제, 얼마, 전, 에서, 셔츠, 주문, 했, 거든요, 고객, 님, 예, ..."


In [56]:
te.head(3)

,tokenize_txt,label,tokenized_del_stopwords
4198,"[네, 쇼핑, 입니다, 네, 여보세요, 네, 고객, 님, 무엇, 을, 도와드릴까요,...",0,"[쇼핑, 입니다, 여보세요, 고객, 님, 무엇, 도와드릴까요, 쇼핑, 에서, 신발,..."
29907,"[은행, 입니다, 무엇, 을, 도와, 드릴까요, 고객, 님, 제, 가, 수신, 거래...",0,"[은행, 입니다, 무엇, 도와, 드릴까요, 고객, 님, 제, 수신, 거래, 골드, ..."
4782,"[쇼핑, 입니다, 무엇, 을, 도와드릴까요, 지금, 청바지, 나오, 는, 거, 하나...",0,"[쇼핑, 입니다, 무엇, 도와드릴까요, 지금, 청바지, 나오, 거, 하나, 주문, ..."


In [57]:
tr.shape, te.shape

((44666, 3), (11167, 3))

In [19]:
tr['label'].value_counts() / len(tr)

0    0.99044
1    0.00956
Name: label, dtype: float64

In [20]:
te['label'].value_counts() / len(te)

0    0.991403
1    0.008597
Name: label, dtype: float64

- split data 저장

In [21]:
tr.to_csv('/content/drive/MyDrive/2022 ADV/data/tr.csv', index=False)
te.to_csv('/content/drive/MyDrive/2022 ADV/data/te.csv', index=False)

- data encoding (tr)

In [22]:
tr['tokenized_del_stopwords_sent'] = tr['tokenized_del_stopwords'].apply(lambda x : ' '.join(x))

In [23]:
tr.drop(['tokenize_txt'], axis=1, inplace=True)

In [24]:
tr.head(3)

,label,tokenized_del_stopwords,tokenized_del_stopwords_sent
4841,0,"[쇼핑, 입니다, 무엇, 도와, 드릴까요, 지금, 있, 거, 주문, 할, 수, 있,...",쇼핑 입니다 무엇 도와 드릴까요 지금 있 거 주문 할 수 있 나요 고객 님 께서 쇼...
35591,0,"[롯데, 월드, 가려, 는데요, 방법, 알려, 주, 세요, 출발지, 어디, 십니까,...",롯데 월드 가려 는데요 방법 알려 주 세요 출발지 어디 십니까 영등포 입니다 도착지...
13194,0,"[입니다, 제, 얼마, 전, 에서, 셔츠, 주문, 했, 거든요, 고객, 님, 예, ...",입니다 제 얼마 전 에서 셔츠 주문 했 거든요 고객 님 예 근데 배송 잘못 되 어서...


- CountVectorizer

In [25]:
tr.shape

(44666, 3)

In [26]:
countvec = CountVectorizer()
cnt_vec = countvec.fit_transform(tr['tokenized_del_stopwords_sent'].tolist()) # train X vector

In [30]:
print('첫 번째 문장 :', tr['tokenized_del_stopwords_sent'].iloc[0])
print('첫 번째 label :', tr['label'].iloc[0])
print('Count vec shape :', cnt_vec.shape, '\n')
print(cnt_vec[0])

첫 번째 문장 : 쇼핑 입니다 무엇 도와 드릴까요 지금 있 거 주문 할 수 있 나요 고객 님 께서 쇼핑 정보 등록 되 어 있 않 으셔 가지 요 정보 등록 다음 주문 도와 드리 겠 습니다 그렇게 할게요 회원 서비스 제공 위해 고객 님 성함 생년월일 주소 연락처 등록 있 습니다 동의 십니까 아 잠깐 만 요 결제 카드 남편 거 거든요 상관 없 으세요 고객 님 아 그래요 회원 정보 제공 동의 십니까 동의 합니다 고객 님 성함 휴대폰 번호 말씀 부탁 드립니다 이고 입니다 생년월일 여섯 자리 말씀 부탁 드립니다 주소 말씀 부탁 드립니다 충남 보령시 고객 님 단독 주택 신 건가요 맞 아요 소중 정보 확인 감사 합니다 이벤트 나 상품 서비스 광고 문자 로 드려도 되 겠 습니까 아니 요 보내 마세요 불편 드리 않 도록 주문 관한 문자 만 발송 도록 겠 습니다 알 겠 어요 현재 방송 중 상품 아스 토니 쉬 곰팡이 제거 세트 오만 팔 백 원 주문 도와 드리 면 될까요 사만 구천 팔 백 원 아닙니까 일시불 결제 시 면 사만 구천 팔 백 원 세요 일시불 로 할게요 카드 번호 확인 부탁 드립니다 관계 남편 신 건가요 맞 아요 남편 분 성함 생년월일 확인 부탁 드립니다 고요 생년월일 말씀 해 주 세요 카드 유효 기간 확인 부탁 드립니다 사만 구천 팔 백 원 일시불 결제 바로 도와 드리 겠 습니다 알 겠 습니다 고객 님 안전 결제 위해서 카드 인증 에이알 에스 로 연결 해 드리 겠 습니다 안 내 멘 트 따라 카드 비밀 번호 앞 두 자리 입력 부탁 드립니다 연결 겠 습니다
첫 번째 label : 0
Count vec shape : (44666, 30398) 

  (0, 15008)	2
  (0, 21819)	2
  (0, 9823)	1
  (0, 6771)	4
  (0, 7443)	1
  (0, 24368)	1
  (0, 23854)	4
  (0, 4503)	1
  (0, 1689)	6
  (0, 4200)	1
  (0, 23174)	4
  (0, 7637)	3
  (0, 20440)	1

In [31]:
countvec.vocabulary_ # 단어장

{'쇼핑': 15008,
 '입니다': 21819,
 '무엇': 9823,
 '도와': 6771,
 '드릴까요': 7443,
 '지금': 24368,
 '주문': 23854,
 '나요': 4503,
 '고객': 1689,
 '께서': 4200,
 '정보': 23174,
 '등록': 7637,
 '으셔': 20440,
 '가지': 307,
 '다음': 5860,
 '드리': 7434,
 '습니다': 15655,
 '그렇게': 3111,
 '할게요': 28874,
 '회원': 30066,
 '서비스': 14003,
 '제공': 23353,
 '위해': 20090,
 '성함': 14400,
 '생년월일': 13850,
 '주소': 23888,
 '연락처': 18378,
 '동의': 7088,
 '십니까': 16429,
 '잠깐': 22231,
 '결제': 1385,
 '카드': 26422,
 '남편': 4742,
 '거든요': 897,
 '상관': 13559,
 '으세요': 20439,
 '그래요': 3032,
 '합니다': 28971,
 '휴대폰': 30225,
 '번호': 11233,
 '말씀': 8924,
 '부탁': 12261,
 '드립니다': 7451,
 '이고': 20748,
 '여섯': 18189,
 '자리': 21993,
 '충남': 26128,
 '보령시': 11614,
 '단독': 5938,
 '주택': 23983,
 '건가요': 1016,
 '아요': 16842,
 '소중': 14803,
 '확인': 29912,
 '감사': 532,
 '이벤트': 20959,
 '상품': 13711,
 '광고': 2364,
 '문자': 9941,
 '드려도': 7419,
 '습니까': 15652,
 '아니': 16676,
 '보내': 11565,
 '마세요': 8675,
 '불편': 12527,
 '도록': 6706,
 '관한': 2352,
 '발송': 10800,
 '어요': 17738,
 '현재': 29532,
 '방송': 10924,
 '아스': 16807

- CountVectorizer 저장

In [33]:
with open('/content/drive/MyDrive/2022 ADV/data/countvec.pkl', 'wb') as f:
    pickle.dump(countvec, f)

In [34]:
with open('/content/drive/MyDrive/2022 ADV/data/countvec.pkl', 'rb') as f:
    load_countvec = pickle.load(f)

- TfidfVectorizer

In [35]:
tfidfvec = TfidfVectorizer()
tfidf_vec = tfidfvec.fit_transform(tr['tokenized_del_stopwords_sent'].tolist())

In [36]:
print('첫 번째 문장 :', tr['tokenized_del_stopwords_sent'].iloc[0])
print('첫 번째 label :', tr['label'].iloc[0])
print('TF-IDF vec shape :', tfidf_vec.shape, '\n')
print(tfidf_vec[0])

첫 번째 문장 : 쇼핑 입니다 무엇 도와 드릴까요 지금 있 거 주문 할 수 있 나요 고객 님 께서 쇼핑 정보 등록 되 어 있 않 으셔 가지 요 정보 등록 다음 주문 도와 드리 겠 습니다 그렇게 할게요 회원 서비스 제공 위해 고객 님 성함 생년월일 주소 연락처 등록 있 습니다 동의 십니까 아 잠깐 만 요 결제 카드 남편 거 거든요 상관 없 으세요 고객 님 아 그래요 회원 정보 제공 동의 십니까 동의 합니다 고객 님 성함 휴대폰 번호 말씀 부탁 드립니다 이고 입니다 생년월일 여섯 자리 말씀 부탁 드립니다 주소 말씀 부탁 드립니다 충남 보령시 고객 님 단독 주택 신 건가요 맞 아요 소중 정보 확인 감사 합니다 이벤트 나 상품 서비스 광고 문자 로 드려도 되 겠 습니까 아니 요 보내 마세요 불편 드리 않 도록 주문 관한 문자 만 발송 도록 겠 습니다 알 겠 어요 현재 방송 중 상품 아스 토니 쉬 곰팡이 제거 세트 오만 팔 백 원 주문 도와 드리 면 될까요 사만 구천 팔 백 원 아닙니까 일시불 결제 시 면 사만 구천 팔 백 원 세요 일시불 로 할게요 카드 번호 확인 부탁 드립니다 관계 남편 신 건가요 맞 아요 남편 분 성함 생년월일 확인 부탁 드립니다 고요 생년월일 말씀 해 주 세요 카드 유효 기간 확인 부탁 드립니다 사만 구천 팔 백 원 일시불 결제 바로 도와 드리 겠 습니다 알 겠 습니다 고객 님 안전 결제 위해서 카드 인증 에이알 에스 로 연결 해 드리 겠 습니다 안 내 멘 트 따라 카드 비밀 번호 앞 두 자리 입력 부탁 드립니다 연결 겠 습니다
첫 번째 label : 0
TF-IDF vec shape : (44666, 30398) 

  (0, 21826)	0.048473789337181286
  (0, 12703)	0.051350123801497234
  (0, 7747)	0.04572879900763162
  (0, 18323)	0.0973780682764801
  (0, 17998)	0.05206488514640023
  (0, 18036)	0.054722

In [37]:
tfidfvec.vocabulary_

{'쇼핑': 15008,
 '입니다': 21819,
 '무엇': 9823,
 '도와': 6771,
 '드릴까요': 7443,
 '지금': 24368,
 '주문': 23854,
 '나요': 4503,
 '고객': 1689,
 '께서': 4200,
 '정보': 23174,
 '등록': 7637,
 '으셔': 20440,
 '가지': 307,
 '다음': 5860,
 '드리': 7434,
 '습니다': 15655,
 '그렇게': 3111,
 '할게요': 28874,
 '회원': 30066,
 '서비스': 14003,
 '제공': 23353,
 '위해': 20090,
 '성함': 14400,
 '생년월일': 13850,
 '주소': 23888,
 '연락처': 18378,
 '동의': 7088,
 '십니까': 16429,
 '잠깐': 22231,
 '결제': 1385,
 '카드': 26422,
 '남편': 4742,
 '거든요': 897,
 '상관': 13559,
 '으세요': 20439,
 '그래요': 3032,
 '합니다': 28971,
 '휴대폰': 30225,
 '번호': 11233,
 '말씀': 8924,
 '부탁': 12261,
 '드립니다': 7451,
 '이고': 20748,
 '여섯': 18189,
 '자리': 21993,
 '충남': 26128,
 '보령시': 11614,
 '단독': 5938,
 '주택': 23983,
 '건가요': 1016,
 '아요': 16842,
 '소중': 14803,
 '확인': 29912,
 '감사': 532,
 '이벤트': 20959,
 '상품': 13711,
 '광고': 2364,
 '문자': 9941,
 '드려도': 7419,
 '습니까': 15652,
 '아니': 16676,
 '보내': 11565,
 '마세요': 8675,
 '불편': 12527,
 '도록': 6706,
 '관한': 2352,
 '발송': 10800,
 '어요': 17738,
 '현재': 29532,
 '방송': 10924,
 '아스': 16807

- TfidfVectorizer 저장

In [38]:
with open('/content/drive/MyDrive/2022 ADV/data/tfidfvec.pkl', 'wb') as f:
    pickle.dump(tfidfvec, f)

In [39]:
with open('/content/drive/MyDrive/2022 ADV/data/tfidfvec.pkl', 'rb') as f:
    load_tfidfvec = pickle.load(f)